In [10]:
import itertools
import math
import numpy as np

In [11]:
def shapley_value(n, v):
    seq = [str(i + 1) for i in range(n)]
    All_set = []
    for i in range(n):
        comb = list(itertools.combinations(seq,i + 1))
        All_set.extend(comb)

    new_All_set = ['0']
    for i in range(len(All_set)): 
        s=""
        a = All_set[i]
        for j in a:
            s += j
        new_All_set.append(s)

    zero_set = list(0 for _ in range(len(new_All_set)))

    S = new_All_set
    V = zero_set
    
    for i in range(len(new_All_set)):
        V[i] = v[i]
        
    sv = []
    for i in range(n):
        res = 0
        i_in = [s for s in S if str(i + 1) in s]
        i_not_in = [s for s in S if str(i + 1) not in s]
        for j in range(len(i_in)):
            res += math.factorial(len(i_in[j]) - 1) * math.factorial(n-len(i_in[j])) / math.factorial(n) \
            * (V[S.index(i_in[j])] - V[S.index(i_not_in[j])])
        sv.append(["player"+str(i+1) ,res])
    return sv

In [12]:
def value_function_price(Q, pv, pa):
    Ns = np.where(Q>=0)
    Nb = np.where(Q<0)
    
    ns = np.sum(Q[Q>=0])
    nb = np.sum(Q[Q<0])
    Ep = ns + nb
    Er = -Ep
    v = pa * np.max([0, Ep]) - pv * np.max([0, Er])
    
    return v

In [13]:
def price(subset, energy, pv):

    comb = sum([list(map(list, itertools.combinations(Q, i))) for i in range(len(Q) + 1)], [])
    comb = np.array([np.array(xi) for xi in comb])
    v = []
    
    for i in range(len(comb)):
        v.append(value_function_price(comb[i], 2, 1))
        
    sh = shapley_value(len(Q),v)
    sh = np.array([np.array(xi[1]) for xi in sh])
    idx = np.array(np.where(sh > 0)).reshape(-1)
    
    if len(idx) > 0:
        psm = sum(sh[sh > 0])/sum(np.array([np.array(xi) for xi in v])[idx+1])
    else:
        psm = pv
        
    print('Coalition: ',subset, ' trading price: ',psm)
    return psm

In [14]:
def L_sm_cps(P_req, d):
    
    a = R*d / U0**2
    b = (beta - 1)
    c = P_req
    
    delta = b**2 - 4*a*c
    if delta < 0:
        return (1-beta)*U0**2 / (2*R*d)
    else:
        x1 = (-b + math.sqrt(delta)) / (2 * a)
        x2 = (-b - math.sqrt(delta)) / (2 * a)
        return np.min([x1,x2])
    
def P_loss_cps(P):
    return R*d * P**2 / (U0)**2 + beta*P

In [15]:
def L_sm_sm(P_req, d):
    
    a = R*d / U1**2
    b = -1
    c = P_req
    
    delta = b**2 - 4*a*c
    if delta < 0:
        return U1**2 / (2*R*d)
    else:
        x1 = (-b + math.sqrt(delta)) / (2 * a)
        x2 = (-b - math.sqrt(delta)) / (2 * a)
        
        return np.min([x1,x2])
    
def P_loss_sm(P, d):
    return R*d * P**2 / (U1)**2

In [16]:
def payoff(subset, Q, energy, psm, u_sell, u):
    Li_cps = 0
    Li_sm = 0
    E_i = Q[Q>=0]
    Q_i = -Q[Q<0]
    
    idx_pos = np.where(energy >= 0)[0]
    idx_neg = np.where(energy < 0)[0]
    
    for i in range(len(idx_pos)-1, -1, -1):
        if all(idx_pos[i] != subset-1):
            idx_pos = np.delete(idx_pos, i, 0)

    for i in range(len(idx_neg)-1, -1,-1):
        if all(idx_neg[i] != subset-1):
            idx_neg = np.delete(idx_neg, i, 0)
            
    
    if all(Q > 0):
        for j in range(len(E_i)):
            u_sell[j] += energy[idx_pos[j]] * pa
        return sum(u_sell)
    

    elif all(Q < 0):
        for i in range(len(Q_i)):
            Li_cps = L_sm_cps(Q_i[i], d[idx_neg[i]])
            u[i] -= Li_cps*pv
        return sum(u)

    else:
        E_i = Q[Q>=0]
        Q_i = -Q[Q<0]
        for i in range(len(Q_i)):
            Li_cps = 0
            Li_sm = 0
            dis = np.sort(distance[idx_neg[i]][idx_pos])
            sort = np.argsort(distance[idx_neg[i],:][subset-1])
            
            Q[Q>0] = E_i
            energy_ordered = Q[sort]
            E_i_aft = energy_ordered[energy_ordered>=0]
            h = 0
            if (E_i != E_i_aft).all():
                u_sell.reverse()
                h = 1
            E_i = E_i_aft

            for j in range(len(E_i)):
                Li_sm = L_sm_sm(Q_i[i], dis[j])
                
                if E_i[j] >= Li_sm:
                    P_loss = P_loss_sm(Li_sm, dis[j])
                    check = float(Q_i[i] - Li_sm + P_loss)
                    E_i[j] -= Li_sm
                    u_sell[j] += Li_sm * psm
                    u[i] -= Li_sm*psm
                    
                    if ((check<= 0.0001) & (check>=-0.0001)):
                        check = 0
                        break
                else:
                    P_loss = P_loss_sm(E_i[j], dis[j])
                    check = float(Q_i[i] - E_i[j] + P_loss)
                    u_sell[j] += E_i[j] * psm
                    E_i[j] = 0
                    u[i] -= E_i[j]*psm
                Q_i[i] = check
                
            if check != 0:
                Li_cps = L_sm_cps(Q_i[i], d[idx_neg[i]])
            
            u[i] -= Li_cps*pv
        for j in range(len(E_i)):
            u_sell[j] += E_i[j] * pa
    if h == 1:
        u_sell.reverse()
    #for i in range(len(E_i)): print('Player: ', idx_pos[i]+1 , ' earns: ', u_sell[i])
    #for i in range(len(Q_i)): print('Player: ', idx_neg[i]+1 , ' pays: ', u[i])
    return sum(u_sell) + sum(u)

def comb(n,k):
    return int(math.factorial(n)/(math.factorial(k)*math.factorial(n-k)))

# Simulation

In [17]:
R = 0.2
beta = 0.02
U0 = 50
U1 = 22

In [19]:
players = [1, 2, 3,4]

# Select the energy
#energy = np.array([335, -495, -96, 541], dtype = float) #simulation
#energy = np.array([-798, -825, -757, -649], dtype = float) #winter
energy = np.array([-335, -495, -96, 541], dtype = float) # summer

pv = 2
pa = 1
d = np.array([4.95, 5.09, 5.87, 5.87])
distance = np.array([[0,0.2,10.82,10.82],[0.2,0,10.97,10.96],[10.82,10.97,0,0.14],[10.82,10.96,0.14,0]])


payoff_list = {
    (0,): 0
}
for L in range(1, len(players)+1):
    for subset in itertools.combinations(players, L):
        v_2 = [0]
        subset = np.array(subset)
        Q = energy[subset-1]
        
        u_sell = [0]*len(Q[Q>0])
        u = [0]*len(Q[Q<0])
        
        psm = price(subset, Q, pv)
        
        if len(subset)>1:
            for h in range(len(payoff_list)):
                if len(list(payoff_list.keys())[h]) < len(subset):
                    for k in range(len(list(payoff_list.keys())[h])):
                        if any(s == list(list(payoff_list.keys())[h])[k] for s in subset):
                            v_2.append(list(payoff_list.values())[h])
                            break
            
        val = payoff(subset, Q, energy, psm, u_sell, u)
        payoff_list[tuple(subset)] = val
        v_2.append(val)
        print('Value function: ', val)
        shap = np.array(shapley_value(len(Q),v_2))[:,1]
        for i in range(len(subset)):
            print('Player: ',subset[i], 'Shapley value: ', shap[i])

Coalition:  [1]  trading price:  2
Value function:  -819.2908368585148
Player:  1 Shapley value:  -819.2908368585148
Coalition:  [2]  trading price:  2
Value function:  -1442.505482499656
Player:  2 Shapley value:  -1442.505482499656
Coalition:  [3]  trading price:  2
Value function:  -206.09507358442295
Player:  3 Shapley value:  -206.09507358442295
Coalition:  [4]  trading price:  1.0
Value function:  541.0
Player:  4 Shapley value:  541.0
Coalition:  [1 2]  trading price:  2
Value function:  -2261.796319358171
Player:  1 Shapley value:  -819.2908368585149
Player:  2 Shapley value:  -1442.505482499656
Coalition:  [1 3]  trading price:  2
Value function:  -1025.3859104429378
Player:  1 Shapley value:  -819.2908368585148
Player:  3 Shapley value:  -206.09507358442295
Coalition:  [1 4]  trading price:  1.3096118299445472
Value function:  -227.51956002180236
Player:  1 Shapley value:  -793.9051984401585
Player:  4 Shapley value:  566.3856384183562
Coalition:  [2 3]  trading price:  2
Val

C:\Users\lucia\AppData\Local\Temp/ipykernel_2300/1606606569.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  comb = np.array([np.array(xi) for xi in comb])
